In [1]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB
import scipy.sparse as sp

In [2]:
# ok, let's say we have two fields we want to schedule

# airmass of pointings at each time (just making things up)
am1 = [1.5, 1.45, 1.4, 1.3, 1.2, 1.1, 1.1, 1.2, 1.3, 1.4]
am2 = [3,    3.5,   3, 2.5, 2.2, 2.1, 1.0, 1.4, 1.3, 1.2]

# mark if a time step is blocked for some reason
open_block = np.ones(10, dtype=bool)
open_block[-2] = False
open_block[2] =False
open_block[1] = False

In [3]:
open_block

array([ True, False, False,  True,  True,  True,  True,  True, False,
        True])

In [4]:
# Define the model
m = gp.Model("try_sched")

Academic license - for non-commercial use only - expires 2021-06-18
Using license file /Users/yoachim/Dropbox/Apps/Gurobi/gurobi.lic


In [5]:
# Add two variables
pointing_1 = m.addMVar(10, vtype=GRB.BINARY, name="pointing_1")
pointing_2 = m.addMVar(10, vtype=GRB.BINARY, name="pointing_2")

# Here's a diagonal matrix with airmass of each field
ams1 = sp.diags(am1)
ams2 = sp.diags(am2)


am1_realized = m.addMVar(10)
m.addConstr(am1_realized == ams1 @ pointing_1)

am2_realized = m.addMVar(10)
m.addConstr(am2_realized == ams2 @ pointing_2)

# Let's force there to be 2 observations for each field
#m.addConstr(pointing_1.sum() == 2)
#m.addConstr(pointing_2.sum() == 2)

# Make sure pointings aren't scheduled in blocked times, and sum to the number of allowed observations
m.addConstr(pointing_1 @ open_block == 2)
m.addConstr(pointing_2 @ open_block == 2)

# Constraint that we can't double-book a time slot
m.addConstr(pointing_1 @ pointing_2 == 0)


<gurobi.QConstr Not Yet Added>

In [6]:
# minimize the total airmass we're taking things at
m.setObjective(am1_realized.sum()+am2_realized.sum(), GRB.MINIMIZE)

In [7]:
m.optimize()

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 22 rows, 40 columns and 54 nonzeros
Model fingerprint: 0x0a24c311
Model has 1 quadratic constraint
Variable types: 20 continuous, 20 integer (20 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Presolve removed 13 rows and 26 columns
Presolve time: 0.00s
Presolved: 9 rows, 14 columns, 28 nonzeros
Variable types: 0 continuous, 14 integer (14 binary)
Found heuristic solution: objective 4.5000000

Root relaxation: cutoff, 3 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0     cutoff    0         4.50000    4.50000  0.00%     -    0s

Explored 0 nodes

In [8]:
# really? In the year of our lord 2021 and we're still naming variables X?
pointing_1.X, pointing_2.X

(array([-0.,  0.,  0., -0.,  1.,  1., -0., -0.,  0., -0.]),
 array([-0.,  0.,  0., -0., -0., -0.,  1., -0.,  0.,  1.]))

In [9]:
open_block

array([ True, False, False,  True,  True,  True,  True,  True, False,
        True])

In [10]:
# maybe I want to maximize the number of times there's a gap in a certain range.
# I can construct some matrix where the reward is based on the gap between sched[i],sched[j]
# then A[i,j] @ sched[i] @ sched[j] maybe?

In [11]:
days = np.arange(10)

In [12]:
m = gp.Model("try_sched")

In [13]:
sched = m.addMVar(10, vtype=GRB.BINARY, name="pointing_1")
m.addConstr(sched.sum() == 3)
# Ah ha! Here's how to set multiple constraints in a loop!
# could use python to do this as something sophisticated like constrain to only one sequence per night.
m.addConstrs(sched[i+1] @ sched[i] == 0 for i in range(9))

cost = np.arange(1, 11)

In [14]:
m.setObjective(sched @ cost, GRB.MINIMIZE)

In [15]:
m.optimize()

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 1 rows, 10 columns and 10 nonzeros
Model fingerprint: 0xd6247462
Model has 9 quadratic constraints
Variable types: 0 continuous, 10 integer (10 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [1e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 3e+00]
Presolve removed 0 rows and 5 columns
Presolve time: 0.00s
Presolved: 1 rows, 5 columns, 5 nonzeros
Variable types: 0 continuous, 5 integer (5 binary)
Found heuristic solution: objective 9.0000000

Root relaxation: cutoff, 0 iterations, 0.00 seconds

Explored 0 nodes (0 simplex iterations) in 0.03 seconds
Thread count was 4 (of 4 available processors)

Solution count 1: 9 

Optimal solution found (tolerance 1.00e-04)
Best objective 9.000000000000e+00, best bound 9.000000000000e+00, gap 0.0000%


In [16]:
sched.X

array([1., 0., 1., 0., 1., 0., 0., 0., 0., 0.])